In [5]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [1]:
!pip install opencv-python==4.5.3.56

In [2]:
!pkg-config --modversion opencv

3.2.0


In [3]:
import cv2
cv2.__version__

'4.5.3'

In [22]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

net = cv2.dnn.readNet("/gdrive/MyDrive/Colab/yolov4.cfg", "/gdrive/MyDrive/Colab/yolov4_15000.weights")
#net = cv2.dnn.readNet("E:/YOLOv4/darknet/cfg/testcfg/yolov4.cfg", "E:/YOLOv4/darknet/cfg/yolov4.weights")
classes = []
with open("/gdrive/MyDrive/Colab/obj.names" , "r") as f:
#with open("E:/YOLOv4/darknet/build/darknet/x64/data/obj.names" , "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in  net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(500,3))

#img = cv2.imread("E:\YOLOv4\pjimage1.jpg")
img = cv2.imread("/gdrive/MyDrive/Colab/garlic_test1.jpg")
#img = cv2.imread("/gdrive/MyDrive/Colab/cabbage_test2.jpg")
#img = cv2.imread("/gdrive/MyDrive/Colab/cucumber_test1.jpg")
#img = cv2.imread("/gdrive/MyDrive/Colab/carrot_test2.jpg")
#img = cv2.imread("/gdrive/MyDrive/Colab/chicken_test10.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
labels = []
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        labels.append(label)
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

labels_set = set(labels)
labels = list(labels_set)
print(labels)
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.